<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#model-selection" data-toc-modified-id="model-selection-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>model selection</a></span><ul class="toc-item"><li><span><a href="#Finding-the-indices-for-KFold" data-toc-modified-id="Finding-the-indices-for-KFold-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Finding the indices for <code>KFold</code></a></span></li></ul></li></ul></div>

# model selection


In [11]:
import sklearn
import numpy as np


## Finding the indices for `KFold`

The class `KFold` allows us to generate the train and validation indicies for performing training and validation indicies with different parts of our data.


In [31]:
from sklearn.model_selection import KFold
X = np.random.randn(100,4)
X.shape

(100, 4)

In [32]:
folds = KFold(10,shuffle=False)
splits = folds.split(X)
for tr_ind,va_ind in splits:
    print(va_ind)

[0 1 2 3 4 5 6 7 8 9]
[10 11 12 13 14 15 16 17 18 19]
[20 21 22 23 24 25 26 27 28 29]
[30 31 32 33 34 35 36 37 38 39]
[40 41 42 43 44 45 46 47 48 49]
[50 51 52 53 54 55 56 57 58 59]
[60 61 62 63 64 65 66 67 68 69]
[70 71 72 73 74 75 76 77 78 79]
[80 81 82 83 84 85 86 87 88 89]
[90 91 92 93 94 95 96 97 98 99]


If shuffle is true then we sample all rows from our dataset randomly

In [33]:

folds = KFold(10,shuffle=True)
splits = folds.split(X)
for tr_ind,va_ind in splits:
    print(va_ind)

[ 0 44 58 61 64 68 77 82 91 92]
[ 9 11 13 18 28 52 60 79 80 86]
[ 4 16 17 21 42 43 49 53 56 98]
[12 31 40 59 66 70 75 90 94 95]
[ 2  3  7  8 46 65 69 84 97 99]
[ 5 10 27 37 45 57 71 72 78 83]
[ 6 25 35 50 55 67 76 87 89 93]
[ 1 22 24 29 36 41 47 63 73 81]
[14 15 19 20 32 39 48 74 88 96]
[23 26 30 33 34 38 51 54 62 85]
